<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
import string

import ebooklib
from ebooklib import epub



### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [71]:
# Armar el dataset
df = pd.read_csv('ConstitucionNacional.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_44151/3648484044.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,CONSTITUCION DE LA NACION ARGENTINA
1,PREÁMBULO
2,Nos los representantes del pueblo de la Nación...
3,PRIMERA PARTE
4,Capítulo Primero


In [72]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 364


### 1 - Preprocesamiento

In [73]:
import stanza
import spacy_stanza
import warnings


# Vamos a usar SpaCy-Stanza. Stanza es una librería de NLP de Stanford
# SpaCy armó un wrapper para los pipelines y modelos de Stanza
# https://stanfordnlp.github.io/stanza/

# Descargar el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

2023-06-14 21:40:11 INFO: Downloading default packages for language: es (Spanish)...
2023-06-14 21:40:12 INFO: File exists: /home/gonzalo/stanza_resources/es/default.zip.
2023-06-14 21:40:15 INFO: Finished downloading models and saved to /home/gonzalo/stanza_resources.
2023-06-14 21:40:15 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2023-06-14 21:40:15 INFO: Use device: cpu
2023-06-14 21:40:15 INFO: Loading: tokenize
2023-06-14 21:40:15 INFO: Loading: mwt
2023-06-14 21:40:15 INFO: Loading: pos
2023-06-14 21:40:15 INFO: Loading: lemma
2023-06-14 21:40:15 INFO: Loading: depparse
2023-06-14 21:40:16 INFO: Loading: ner
2023-06-14 21:40:17 INFO: Done loading processors!


In [74]:
import re
import unicodedata

# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):    
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' # igual al anterior pero sin cifras numéricas
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [75]:
#Silencio los UserWarning de Stanza
warnings.filterwarnings("ignore", category=UserWarning)

sentence_tokens = []

# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras 
for _, row in df[:None].iterrows():
    tokens = nlp(preprocess_clean_text(row[0].lower()))
    # lematizar los tokens
    tokens_lemma = []
    for token in tokens:            
        tokens_lemma.append(token.lemma_)
    sentence_tokens.append(tokens_lemma)
warnings.resetwarnings()

In [76]:
# Demos un vistazo
for i in range(4):
    print(f'Documento {i+1}:')
    print (df[0][i])
    print(f'-->{sentence_tokens[i]}\n')


Documento 1:
CONSTITUCION DE LA NACION ARGENTINA
-->['constitucion', 'de', 'el', 'nacion', 'argentino']

Documento 2:
PREÁMBULO
-->['preambulo']

Documento 3:
Nos los representantes del pueblo de la Nación Argentina, reunidos en Congreso General Constituyente por voluntad y elección de las provincias que la componen, en cumplimiento de pactos preexistentes, con el objeto de constituir la unión nacional, afianzar la justicia, consolidar la paz interior, proveer a la defensa común, promover el bienestar general, y asegurar los beneficios de la libertad, para nosotros, para nuestra posteridad, y para todos los hombres del mundo que quieran habitar en el suelo argentino: invocando la protección de Dios, fuente de toda razón y justicia: ordenamos, decretamos y establecemos esta Constitución, para la Nación Argentina.
-->['yo', 'el', 'representante', 'de', 'el', 'pueblo', 'de', 'el', 'nacion', 'argentino', 'reunido', 'en', 'congreso', 'general', 'constituyente', 'por', 'voluntad', 'y', 'elec

En los documentos evaluados arriba se pueden ver que la lematizacion hizo multiples modificaciones como ser:
- Cambio de genero en la palabra "la" por "el" en el documento 1
- Cambio de numero en la palabra "nos" por "yo" en el documento 3
- Cambio de numero en la palabra "representantes" por "representante" en el documento 3
- Cambio de genero en la palabra "primera" por "primero" en el documento 4

In [77]:
#check unique words in the corpus
unique_words = set()
for sentence in sentence_tokens:
    unique_words.update(sentence)

print("Cantidad de palabras únicas:", len(unique_words))


Cantidad de palabras únicas: 1731


### 2 - Crear los vectores (word2vec)

In [78]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [79]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [80]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [81]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 364


In [82]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 357


### 3 - Entrenar el modelo generador

In [85]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 38791.89453125
Loss after epoch 1: 39918.98828125
Loss after epoch 2: 40247.8125
Loss after epoch 3: 39012.5703125
Loss after epoch 4: 39443.046875
Loss after epoch 5: 39697.421875
Loss after epoch 6: 39697.171875
Loss after epoch 7: 40179.21875
Loss after epoch 8: 39861.3125
Loss after epoch 9: 39962.25
Loss after epoch 10: 38639.3125
Loss after epoch 11: 39235.34375
Loss after epoch 12: 40227.0
Loss after epoch 13: 39511.03125
Loss after epoch 14: 39297.5
Loss after epoch 15: 39610.4375
Loss after epoch 16: 37648.0625
Loss after epoch 17: 38810.4375
Loss after epoch 18: 38698.0
Loss after epoch 19: 37251.1875
Loss after epoch 20: 39364.0
Loss after epoch 21: 39053.0625
Loss after epoch 22: 38433.6875
Loss after epoch 23: 38904.8125
Loss after epoch 24: 38991.125
Loss after epoch 25: 37962.4375
Loss after epoch 26: 38187.125
Loss after epoch 27: 37549.875
Loss after epoch 28: 36659.0
Loss after epoch 29: 36631.625
Loss after epoch 30: 36413.875
Loss after epoch 31:

(1032631, 2580600)

### 4 - Ensayar

In [106]:
# Palabras que MÁS se relacionan con "corte":
w2v_model.wv.most_similar(positive=["corte"], topn=5)

[('supremo', 0.5759741067886353),
 ('juramento', 0.5323599576950073),
 ('tribunal', 0.41436323523521423),
 ('inferior', 0.41192734241485596),
 ('prestar', 0.3981272876262665)]

In [87]:
# Palabras menos relacionada con "corte":
w2v_model.wv.most_similar(negative=["corte"])

[('quedar', 0.011580086313188076),
 ('nacional', 0.0067589497193694115),
 ('discriminacion', -0.005624289624392986),
 ('competencia', -0.012568368576467037),
 ('distrito', -0.019746527075767517),
 ('publico', -0.027713028714060783),
 ('referido', -0.028855131939053535),
 ('estado', -0.03367186710238457),
 ('cualquiera', -0.036632079631090164),
 ('pasar', -0.03670481592416763)]

In [94]:
# Palabras que MÁS se relacionan con "voto":
w2v_model.wv.most_similar(positive=["voto"], topn=5)

[('mayoria', 0.44905751943588257),
 ('tercio', 0.42317795753479004),
 ('totalidad', 0.4144941568374634),
 ('sancionado', 0.4045528173446655),
 ('total', 0.3951209485530853)]

In [95]:
# Palabras menos relacionada con "voto":
w2v_model.wv.most_similar(negative=["voto"])

[('asegurar', 0.10534435510635376),
 ('constitucion', 0.06969275325536728),
 ('bien', 0.04718245938420296),
 ('decretar', 0.025744568556547165),
 ('proveer', 0.019139651209115982),
 ('exigir', -0.0014081337722018361),
 ('legislacion', -0.007092334795743227),
 ('medida', -0.007743945345282555),
 ('educacion', -0.009025159291923046),
 ('provincial', -0.0143263079226017)]

Se puede ver que el embedding asocia la palabra corte con terminos relacionados al poder judicial y la palabra voto con el proceso de sancion de una ley.

### 5 - Visualizar agrupación de vectores

In [88]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [89]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

### Palabras relacionadas con el termino corte

<img src="Corte.png" width="1000" align="center">

### Palabras relacionadas con el termino voto

<img src="Voto.png" width="1000" align="center">


# Conclusiones:

- Se entreno un embedding con el corpus de la constitución nacional en español por lo que se utilizo Spacy-Stanza para la tokenizacion y lematizacion del corpus.
- Se entreno un embedding con Gensim que es capas de asociar palabras con terminos relacionados a los procesos democraticos descriptos en el corpus y a diferentes poderes del estado.
